# Naive Bayes project

In [133]:
!pip install nltk


[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: pip install --upgrade pip


In [134]:
!pip install pandas
!pip install sklearn
!pip install matplotlib



[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: pip install --upgrade pip


In [135]:
# Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import pandas as pd
import numpy as np
import unicodedata
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
import pickle
import nltk
from nltk.corpus import stopwords

In [136]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/gitpod/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [137]:
# Read csv

df_raw = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews_dataset.csv')

In [138]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   package_name  891 non-null    object
 1   review        891 non-null    object
 2   polarity      891 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 21.0+ KB


In [139]:
df_raw.sample(10)

,package_name,review,polarity
106,com.linkedin.android,why can't i share my achievements? recently d...,0
5,com.facebook.katana,idk i can't edit my posts? things such as my ...,0
473,com.whatsapp,not working at all after updating..hangs pho...,0
386,com.google.android.talk,"bugs, bugs, and more bugs with each update. i...",0
590,com.evernote,ătoo slow loading webclips!!! ăno pdf su...,0
460,com.whatsapp,annoyed whatsapp used to work great!! but wit...,0
649,com.uc.browser.en,brilliant app brilliant app for browsing and ...,1
642,com.uc.browser.en,"neat nice ,smooth ,speed but downloading is l...",0
836,com.hamropatro,usefull no others app like this....,1
32,com.facebook.katana,i have had to uninstall this app twice this ...,0


In [140]:
df_raw['polarity'].value_counts()

0    584
1    307
Name: polarity, dtype: int64

In [141]:
df_raw.iloc[675,]

package_name                                    com.hamrokeyboard
review           loved it ŕ¤żŕľ app ŕ¤§ŕľŕ¤°ŕľ ŕ¤ŕ¤şŕ¤żŕľ...
polarity                                                        1
Name: 675, dtype: object

**1. Transform dataframe**

We have three columns: package name, review and polarity (0 = bad, 1 = good) Preprocess the data by eliminating the package name column and putting all reviews in lower case.

In [142]:
df_transf = df_raw.copy()

In [143]:
# Drop package_name column
# 
df_transf = df_transf.drop('package_name', axis=1)

In [144]:

df_transf['review'] = df_transf['review'].str.strip()
# elimina espacio libre al principio y al final

In [145]:
# column review to lower case
df_transf['review'] = df_transf['review'].str.lower()

In [146]:
stop = stopwords.words('english')

def clean_words(review):
    if review is not None:
        words = review.strip().split()
        new_words = []
        for word in words:
            if word not in stop:new_words.append(word)
        Result = ' '.join(new_words)    
    else:
        Result = None
    return Result


In [147]:
df_transf['review'] = df_transf['review'].apply(clean_words)

In [148]:
df_transf['review'].str.split(expand=True).stack().value_counts()[:60]

app         317
can't       137
new         136
update      136
like        135
fix         133
please      131
good        128
game        121
one         119
use         118
get         112
even        101
great        99
time         93
really       82
would        80
back         78
still        78
love         77
browser      76
make         74
see          69
using        68
work         67
i'm          67
5            67
open         65
want         64
it.          64
google       63
much         61
phone        61
every        61
u            58
option       58
version      57
go           56
way          56
app.         56
used         55
also         55
android      54
download     53
best         52
.            51
give         50
better       50
play         50
send         49
people       48
able         47
message      47
since        46
many         45
keep         45
works        44
i've         44
old          43
find         43
dtype: int64

In [149]:
def normalize_str(text_string):
    if text_string is not None:
        result=unicodedata.normalize('NFD',text_string).encode('ascii','ignore').decode()
    else:
        result=None 
    return result

In [150]:
df_transf['review']=df_transf['review'].apply(normalize_str)
df_transf['review']=df_transf['review'].str.replace('!','')
df_transf['review']=df_transf['review'].str.replace(',','')
df_transf['review']=df_transf['review'].str.replace('&','')
df_transf['review']=df_transf['review'].str.normalize('NFKC')
df_transf['review']=df_transf['review'].str.replace(r'([a-zA-Z])\1{2,}',r'\1',regex=True)

In [151]:
df_transf.sample(10)

,review,polarity
756,changeable really change way getting news,1
485,best app can't believe free treat paid it.,1
465,backing messages dnt know even added message b...,0
338,option separate viber contacts? see viber cont...,0
251,unbelievably bad support google products. hard...,0
881,game ruined ads. felt like re-downloading play...,0
337,good remove contacts please... lot ppl hate so...,0
25,can't install (error code: -505) samsung galax...,0
665,hamro keyboard fantastic app mobile write nepa...,1
634,nice update really nice update update awesome ...,1


In [152]:
df = df_transf.copy()

**2. Split data frame**

Separate target from feature, and split your data.

In [153]:
X = df['review']
y = df['polarity']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=25)


In [154]:
y_train.value_counts()

0    438
1    230
Name: polarity, dtype: int64

In [155]:
y_test.value_counts()

0    146
1     77
Name: polarity, dtype: int64

**3. Pipeline with two pre-processing steps and one modeling step**

Vectorize your features and use Naive Bayes to classify the reviews as good or bad. We will not focus on hypertuning our model this time. This was an introduction project to sentiment analysis using Naive Bayes.

In [156]:
vect = CountVectorizer() # vector de conteo

text_vec = vect.fit_transform(X_train)

In [157]:
vect.get_feature_names_out() #da nombres de las columnas

array(['000', '10', '100', ..., 'zespole', 'zoom', 'zooming'],
      dtype=object)

In [158]:
# text_vec.toarray()[0]

In [159]:
vect_tfidf = TfidfVectorizer()

text_vec_tfidf = vect_tfidf.fit_transform(X_train)

In [160]:

#np.set_printoptions(threshold=sys.maxsize)
#text_vec_tfidf.toarray()[0]


In [161]:
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer())])
text_norm = text_clf.fit_transform(X_train)

In [162]:
# text_norm.toarray()[0]

In [163]:
clf_1 = MultinomialNB()

clf_1.fit(text_vec, y_train)

MultinomialNB()

In [164]:
clf_1.score(text_vec, y_train)

0.9625748502994012

In [165]:
clf_2 = MultinomialNB()

clf_2.fit(text_vec_tfidf, y_train)

MultinomialNB()

In [166]:
clf_2.score(text_vec_tfidf, y_train)

0.875748502994012

In [167]:
clf_3 = MultinomialNB()

clf_3.fit(text_norm, y_train)

MultinomialNB()

In [168]:
clf_3.score(text_norm, y_train)

0.875748502994012

In [169]:
pred_1 = clf_1.predict(vect.transform(X_test))
pred_2 = clf_2.predict(vect_tfidf.transform(X_test))
pred_3 = clf_3.predict(text_clf.transform(X_test))

print(classification_report(y_test, pred_1))
print(classification_report(y_test, pred_2))
print(classification_report(y_test, pred_3))

              precision    recall  f1-score   support

           0       0.83      0.93      0.88       146
           1       0.83      0.65      0.73        77

    accuracy                           0.83       223
   macro avg       0.83      0.79      0.81       223
weighted avg       0.83      0.83      0.83       223

              precision    recall  f1-score   support

           0       0.71      0.99      0.83       146
           1       0.90      0.23      0.37        77

    accuracy                           0.73       223
   macro avg       0.80      0.61      0.60       223
weighted avg       0.78      0.73      0.67       223

              precision    recall  f1-score   support

           0       0.71      0.99      0.83       146
           1       0.90      0.23      0.37        77

    accuracy                           0.73       223
   macro avg       0.80      0.61      0.60       223
weighted avg       0.78      0.73      0.67       223



In [170]:
# Acá todo junto en un pipeline

text_clf_2 = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf_2.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

**4. Check results**

In [171]:
y_pred = text_clf_2.predict(X_test)
precision_recall_fscore_support(y_test, y_pred, average='weighted')

(0.7751861096997946, 0.726457399103139, 0.6684246410125338, None)

In [172]:
print('Naive Bayes Train Accuracy = ',metrics.accuracy_score(y_train,text_clf_2.predict(X_train)))
print('Naive Bayes Test Accuracy = ',metrics.accuracy_score(y_test,text_clf_2.predict(X_test)))

Naive Bayes Train Accuracy =  0.875748502994012
Naive Bayes Test Accuracy =  0.726457399103139


**5. Randomized search to select hyperparameters**

In [173]:
n_iter_search = 5
parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3)}
gs_clf = RandomizedSearchCV(text_clf_2, parameters, n_iter = n_iter_search)
gs_clf.fit(X_train, y_train)

RandomizedSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                             ('tfidf', TfidfTransformer()),
                                             ('clf', MultinomialNB())]),
                   n_iter=5,
                   param_distributions={'clf__alpha': (0.01, 0.001),
                                        'tfidf__use_idf': (True, False),
                                        'vect__ngram_range': [(1, 1), (1, 2)]})

In [174]:
gs_clf.best_params_

{'vect__ngram_range': (1, 2), 'tfidf__use_idf': False, 'clf__alpha': 0.01}

In [175]:
print('Naive Bayes Train Accuracy (grid random search) = ',metrics.accuracy_score(y_train,gs_clf.predict(X_train)))
print('Naive Bayes Test Accuracy (grid random search) = ',metrics.accuracy_score(y_test,gs_clf.predict(X_test)))

Naive Bayes Train Accuracy (grid random search) =  1.0
Naive Bayes Test Accuracy (grid random search) =  0.8430493273542601


In [176]:
text_clf_count_vect = Pipeline([('vect', CountVectorizer()), ('clf', MultinomialNB())])
text_clf_count_vect.fit(X_train, y_train)


n_iter_search = 5
parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'clf__alpha': (1e-2, 1e-3)}
gs_count_vect = RandomizedSearchCV(text_clf_count_vect, parameters, n_iter = n_iter_search)
gs_count_vect.fit(X_train, y_train)

/home/gitpod/.pyenv/versions/3.8.13/lib/python3.8/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 4 is smaller than n_iter=5. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                             ('clf', MultinomialNB())]),
                   n_iter=5,
                   param_distributions={'clf__alpha': (0.01, 0.001),
                                        'vect__ngram_range': [(1, 1), (1, 2)]})

In [177]:
gs_count_vect.best_params_

{'vect__ngram_range': (1, 1), 'clf__alpha': 0.01}

In [178]:
print('Naive Bayes Train Accuracy (grid random search) = ',metrics.accuracy_score(y_train,gs_count_vect.predict(X_train)))
print('Naive Bayes Test Accuracy (grid random search) = ',metrics.accuracy_score(y_test,gs_count_vect.predict(X_test)))

Naive Bayes Train Accuracy (grid random search) =  0.9940119760479041
Naive Bayes Test Accuracy (grid random search) =  0.820627802690583


In [179]:
y_pred_mejor = gs_clf.predict(X_test)

print(classification_report(y_test, y_pred_mejor))

              precision    recall  f1-score   support

           0       0.82      0.97      0.89       146
           1       0.90      0.61      0.73        77

    accuracy                           0.84       223
   macro avg       0.86      0.79      0.81       223
weighted avg       0.85      0.84      0.83       223



In [180]:
best_model = gs_clf.best_estimator_
best_model

Pipeline(steps=[('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer(use_idf=False)),
                ('clf', MultinomialNB(alpha=0.01))])

Use the app.py to create your pipeline.

Save your naive bayes classification model in the 'models' folder.

In [181]:
# Save best model

pickle.dump(best_model, open('../models/best_model.pickle', 'wb'))

In [182]:
modelo = pickle.load(open('../models/best_model.pickle', 'rb')) # lo leemos
modelo.predict(X_test) # lo usamos para predecir nueva X_test

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0])

In [183]:
# se guarda el modelo
filename = '../models/nb_model.sav'
pickle.dump(modelo, open(filename,'wb'))